In [1]:
import numpy as np 
import pandas as pd 
import datetime
from datetime import datetime
from tqdm import tqdm_notebook as tqdm
import os


from multiprocessing import Pool, TimeoutError, Lock, Value, current_process, cpu_count
from functools import partial
import ALL_STOCK_preprocess_function as func

In [ ]:
file_path = 'D:\\庫存健診開發\\Stock_Crawl\\data\\'
file_list = [file_path + f for f in os.listdir(file_path)]
df_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(func.data_preprocess, file_list), total=len(file_list)), 1):
                df_list.append(x)
                
data = pd.concat(df_list, axis=0)
data['ts'] = pd.to_datetime(data['ts'])
data

In [ ]:
timedf = pd.DataFrame(data['ts'].unique(), columns=['ts'])
timedf = timedf.sort_values(by='ts')
timedf

In [ ]:
df_list = [group[1] for group in data.groupby(data['StockNo'])]

output_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.FillMissingTime, timedf=timedf), df_list), total=len(df_list)), 1):
                if x[0]:
                    output_list.append(x[1])
                else:    
                    continue
                
df = pd.concat(output_list, axis=0)
df

In [ ]:
stock_list = func.GetStockList(df)
stock_list

In [ ]:
df_stock = df[df.StockNo.isin(stock_list)]
df_stock

In [ ]:
df.to_csv('D:\\Strategic_Trading\\PriceForecast\\data\\Stock\\TWSE_ALL_preserve.csv', index=False)

In [2]:
date_df = func.get_stock()
date_df

It cost 1.620126 sec
Data length: 135123
Index(['STOCK_NO', 'CFC14', 'CFC61'], dtype='object')


,StockNo,On_Date,Off_Date
0,Y7003,20190225,20190225
1,000002,20110106,00000000
2,0001,19900218,20050531
3,0015,19930326,20140219
4,0017,19930410,00000000
...,...,...,...
135118,YY0089,19980826,20030101
135119,YY0090,19971218,20010124
135120,YY0091,19980518,20020417
135121,YY00UO,19901111,20090504


In [4]:
date_df.to_csv('D:\\庫存健診開發\\data\\Stock\\Date.csv', index=False)

In [3]:
df = pd.merge(data, date_df, on='StockNo', how='left')
df

,StockNo,On_Date,Off_Date
73657,1101,19620209,00000000


In [ ]:
df = df[df['Off_Date'] != 20191014]
df

In [ ]:
df.loc[df[df['StockNo'] == '1410'].index.tolist(), 'On_Date'] = 20000101
df.loc[df[df['StockNo'] == '1423'].index.tolist(), 'On_Date'] = 20000101
df.loc[df[df['StockNo'] == '1439'].index.tolist(), 'On_Date'] = 20000101
df.loc[df[df['StockNo'] == '2029'].index.tolist(), 'On_Date'] = 20000101

In [ ]:
df['On_Date'] = df['On_Date'].apply(lambda x: str(x)[:4] + '-' + str(x)[4:6] + '-' + str(x)[6:])
df['On_Date'] = pd.to_datetime(df['On_Date'])
df

In [ ]:
d = df[df['ts'].dt.date >= df['On_Date'].dt.date]
d = d.drop(columns=['Off_Date'])
d

In [21]:
date_df = pd.read_csv('D:\\庫存健診開發\\data\\Stock\\new_listing.csv', converters={'StockNo': str})
date_df

,StockNo,OnDate
0,4571,108.09.17
1,2233,108.09.17
2,4572,108.09.10
3,4576,108.09.04
4,5283,108.05.24
...,...,...
635,2445,90.04.02
636,1729,90.03.21
637,9939,90.03.02
638,9938,90.01.12


In [22]:
date_df['OnDate'] = date_df['OnDate'].apply(lambda x: datetime.strptime(str(int(x[:-6]) + 1911) + '-' + x[-5:-3] + '-' + x[-2:], '%Y-%m-%d').date())
date_df

,StockNo,OnDate
0,4571,2019-09-17
1,2233,2019-09-17
2,4572,2019-09-10
3,4576,2019-09-04
4,5283,2019-05-24
...,...,...
635,2445,2001-04-02
636,1729,2001-03-21
637,9939,2001-03-02
638,9938,2001-01-12


In [33]:
df = pd.merge(df, date_df, on='StockNo', how='left')
df

,StockName,StockNo,close,high,low,open,return,total,ts,vol,On_Date,OnDate
0,台泥,1101,19.30,19.90,19.10,19.90,NaN,8.756613e+08,2004-02-11,44990547.0,1962-02-09,NaN
1,台泥,1101,18.70,19.60,18.40,19.40,-0.60,7.295642e+08,2004-02-12,38283630.0,1962-02-09,NaN
2,台泥,1101,20.00,20.00,18.90,19.00,1.30,1.200960e+09,2004-02-13,60809947.0,1962-02-09,NaN
3,台泥,1101,21.00,21.40,20.00,20.00,1.00,3.204683e+09,2004-02-16,154068407.0,1962-02-09,NaN
4,台泥,1101,21.00,21.30,20.40,21.00,0.00,1.477319e+09,2004-02-17,70709589.0,1962-02-09,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3008119,慶豐富,9935,34.50,34.80,33.50,33.85,0.75,6.743481e+07,2019-09-17,1962367.0,2000-09-11,NaN
3008120,慶豐富,9935,34.65,35.20,34.00,34.50,0.15,1.134111e+08,2019-09-18,3261600.0,2000-09-11,NaN
3008121,慶豐富,9935,34.95,35.20,34.70,34.80,0.30,5.601946e+07,2019-09-19,1604885.0,2000-09-11,NaN
3008122,慶豐富,9935,34.15,35.45,34.15,35.05,-0.80,7.139641e+07,2019-09-20,2046476.0,2000-09-11,NaN


In [46]:
d1 = df[df.OnDate.notnull()]
d1['OnDate'] = pd.to_datetime(d1['OnDate'])
d1 = d1[d1['ts'].dt.date >= d1['OnDate'].dt.date]
d1['On_Date'] = d1['OnDate']
d1

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,StockName,StockNo,close,high,low,open,return,total,ts,vol,On_Date,OnDate
88918,宏亞,1236,20.7,21.7,20.5,21.6,NaN,22967407.0,2004-02-11,1091039.0,2001-09-17,2001-09-17
88919,宏亞,1236,20.3,21.1,20.1,20.9,-0.4,14775220.0,2004-02-12,718001.0,2001-09-17,2001-09-17
88920,宏亞,1236,20.7,20.8,20.3,20.3,0.4,7085618.0,2004-02-13,343100.0,2001-09-17,2001-09-17
88921,宏亞,1236,20.9,21.3,20.6,20.7,0.2,23544077.0,2004-02-16,1122155.0,2001-09-17,2001-09-17
88922,宏亞,1236,21.0,21.2,20.6,21.2,0.1,10351183.0,2004-02-17,495839.0,2001-09-17,2001-09-17
...,...,...,...,...,...,...,...,...,...,...,...,...
3004253,世紀鋼,9958,77.3,78.9,77.1,78.9,-0.2,237190800.0,2019-09-17,3050500.0,2008-03-12,2008-03-12
3004254,世紀鋼,9958,77.7,79.3,77.5,77.9,0.4,464879118.0,2019-09-18,5940537.0,2008-03-12,2008-03-12
3004255,世紀鋼,9958,76.3,78.4,76.2,78.4,-1.4,308696223.0,2019-09-19,4011923.0,2008-03-12,2008-03-12
3004256,世紀鋼,9958,75.8,77.0,75.8,76.6,-0.5,132881305.0,2019-09-20,1742684.0,2008-03-12,2008-03-12


In [48]:
d1[d1['StockNo'] == '6669']

,StockName,StockNo,close,high,low,open,return,total,ts,vol,On_Date,OnDate
2728928,緯穎,6669,378.0,387.0,362.0,370.0,-51.5,2.673135e+09,2019-03-27,7122919.0,2019-03-27,2019-03-27
2728929,緯穎,6669,390.5,391.5,374.0,375.0,12.5,1.740783e+09,2019-03-28,4537483.0,2019-03-27,2019-03-27
2728930,緯穎,6669,392.5,398.0,390.0,393.0,2.0,1.230862e+09,2019-03-29,3122075.0,2019-03-27,2019-03-27
2728931,緯穎,6669,381.0,397.5,380.5,396.5,-11.5,1.104980e+09,2019-04-01,2862326.0,2019-03-27,2019-03-27
2728932,緯穎,6669,381.5,383.5,371.5,381.0,0.5,1.182467e+09,2019-04-02,3130654.0,2019-03-27,2019-03-27
...,...,...,...,...,...,...,...,...,...,...,...,...
2729046,緯穎,6669,448.5,453.5,444.0,449.0,2.0,5.268048e+08,2019-09-17,1172825.0,2019-03-27,2019-03-27
2729047,緯穎,6669,451.5,454.5,447.5,448.0,3.0,4.172568e+08,2019-09-18,923775.0,2019-03-27,2019-03-27
2729048,緯穎,6669,440.0,464.5,437.5,459.0,-11.5,1.324651e+09,2019-09-19,2940476.0,2019-03-27,2019-03-27
2729049,緯穎,6669,467.0,468.5,445.0,445.5,27.0,3.074019e+09,2019-09-20,6645297.0,2019-03-27,2019-03-27


In [49]:
d2 = df[df.OnDate.isnull()]
d2

,StockName,StockNo,close,high,low,open,return,total,ts,vol,On_Date,OnDate
0,台泥,1101,19.30,19.90,19.10,19.90,NaN,8.756613e+08,2004-02-11,44990547.0,1962-02-09,NaN
1,台泥,1101,18.70,19.60,18.40,19.40,-0.60,7.295642e+08,2004-02-12,38283630.0,1962-02-09,NaN
2,台泥,1101,20.00,20.00,18.90,19.00,1.30,1.200960e+09,2004-02-13,60809947.0,1962-02-09,NaN
3,台泥,1101,21.00,21.40,20.00,20.00,1.00,3.204683e+09,2004-02-16,154068407.0,1962-02-09,NaN
4,台泥,1101,21.00,21.30,20.40,21.00,0.00,1.477319e+09,2004-02-17,70709589.0,1962-02-09,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3008119,慶豐富,9935,34.50,34.80,33.50,33.85,0.75,6.743481e+07,2019-09-17,1962367.0,2000-09-11,NaN
3008120,慶豐富,9935,34.65,35.20,34.00,34.50,0.15,1.134111e+08,2019-09-18,3261600.0,2000-09-11,NaN
3008121,慶豐富,9935,34.95,35.20,34.70,34.80,0.30,5.601946e+07,2019-09-19,1604885.0,2000-09-11,NaN
3008122,慶豐富,9935,34.15,35.45,34.15,35.05,-0.80,7.139641e+07,2019-09-20,2046476.0,2000-09-11,NaN


In [51]:
d = pd.concat([d2, d1], axis=0)
d = d.drop(columns=['OnDate'])
d

,StockName,StockNo,close,high,low,open,return,total,ts,vol,On_Date
0,台泥,1101,19.3,19.9,19.1,19.9,NaN,8.756613e+08,2004-02-11,44990547.0,1962-02-09
1,台泥,1101,18.7,19.6,18.4,19.4,-0.6,7.295642e+08,2004-02-12,38283630.0,1962-02-09
2,台泥,1101,20.0,20.0,18.9,19.0,1.3,1.200960e+09,2004-02-13,60809947.0,1962-02-09
3,台泥,1101,21.0,21.4,20.0,20.0,1.0,3.204683e+09,2004-02-16,154068407.0,1962-02-09
4,台泥,1101,21.0,21.3,20.4,21.0,0.0,1.477319e+09,2004-02-17,70709589.0,1962-02-09
...,...,...,...,...,...,...,...,...,...,...,...
3004253,世紀鋼,9958,77.3,78.9,77.1,78.9,-0.2,2.371908e+08,2019-09-17,3050500.0,2008-03-12 00:00:00
3004254,世紀鋼,9958,77.7,79.3,77.5,77.9,0.4,4.648791e+08,2019-09-18,5940537.0,2008-03-12 00:00:00
3004255,世紀鋼,9958,76.3,78.4,76.2,78.4,-1.4,3.086962e+08,2019-09-19,4011923.0,2008-03-12 00:00:00
3004256,世紀鋼,9958,75.8,77.0,75.8,76.6,-0.5,1.328813e+08,2019-09-20,1742684.0,2008-03-12 00:00:00


In [52]:
d.to_csv('D:\\庫存健診開發\\data\\Stock\\TWSE_Stock.csv', index=False)